### Ingestion del archivo "person.json"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de spark

In [0]:
from pyspark.sql.types import *

In [0]:
# definimos el schema

name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
persons_schema = StructType(fields=[
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])

In [0]:
persons_df = spark.read \
            .schema(persons_schema)\
            .json(f"{bronze_folder_path}/{v_file_date}/person.json")

#### Paso 2 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "environment"

1. "personId" renombrar a "person_id"
2. Agregar las columnas "ingestion_date" y "environment"
3. Agregar la columna "name" a partir de la concatenacion de "forename" y "surname"

In [0]:
from pyspark.sql.functions import *

In [0]:
persons_with_columns_df = add_ingestion_date(persons_df) \
                    .withColumnRenamed("personId", "person_id") \
                    .withColumn("environment", lit(v_environment)) \
                    .withColumn("name", concat(col("personName.forename"), lit(" "), col("personName.surname"))) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
display(persons_with_columns_df)

#### Paso 3 - Eliminar las columnas no requeridas

In [0]:
persons_final_df = persons_with_columns_df.drop(col("personName"))

In [0]:
display(persons_final_df)

#### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(persons_final_df, "movie_silver", "persons", "file_date")

In [0]:
merge_condition = 'tgt.person_id = src.person_id AND tgt.file_date = src.file_date'
merge_delte_lake(persons_final_df, "movie_silver", "persons", silver_folder_path, merge_condition, "file_date")


In [0]:
%sql
SELECT file_date, count(1)
FROM movie_silver.persons
GROUP BY file_date;

In [0]:
%sql
select * from movie_silver.persons

In [0]:
%fs
ls /mnt/moviehistory2025/silver/persons

In [0]:
dbutils.notebook.exit("success")